## Problem Statement
Over the relatively short 118 year history of powered human flight, the human race has achieved the unachievable time and again. From the first slow short hops of the Wright brothers in 1903 to supersonic jet flight only 44 years later, and to the moon only 22 years after that, untold numbers of people have perished in the advancement of aviation technology. These days, we think almost nothing of boarding a machine that can fly us at altitudes approaching 40,000 feet at almost the speed of sound to get us from one side of the country to the other in mere hours. While the technology is proven and safe enough for us to not worry about the miniscule chance of a serious problem resulting in a devastating outcome, those problems do still arise. There are many causes for aviation mishaps and accidents, and whenever one occurs, investigators gather every possible data point that could lead to a better understanding of what happened and how to prevent it in the future.


## Data Sets
For this exercise, I will be analyzing aviation mishap data sets provided by the Federal Aviation Administration (FAA) Aviation Safety Information Analysis and Sharing (ASIAS) system. This system provides links to multiple source databases provided from the FAA, the National Transportation Safety Board (NTSB), the Bureau of Transportation Statistics (BTS), and the National Aeronautics and Space Administration (NASA), among others, that track various safety issues and incident and accident reports.

## Sample Questions I Seek to Answer
I will begin with exploring the data and producing summary statistics .

Questions to answer:
What are the top 10 most common accidents or incidents?
Of the most common events, what are the top general causes?
What is the most deadly event type? Which results in the most injuries?
What are the top primary causal factors for the most fatal and injurious events?

Does the level of pilot qualification make a difference?
Does the pilot's experience? (measured in total pilot time) Age?


More advanced and difficult questions I'd like to answer if I can:
Are there significant groupings of primary and secondary contributing factors in aviation mishaps when looking at different types of mishaps? For example, do stall or spin mishap types occur more frequently when there is a specific combination of contributing factors (e.g., pilot-based features, aircraft-based features, weather-based features)?

This would suggest that there are specific contributing factors (or groups of factors) that, if focused on via training or other resources, would have a more significant impact on aviation mishap rates for the most common or most severe/fatalmishap types.


## Challenges
1. There are several data sets provided from multiple organizations that are each very large. This required a thorough study of each data set thoroughly to determine the best combination of data and questions to ask.

2. The analysis is further complicated by the fact that the main data set refers to multiple other tables that interpret the various codes recorded in the main table. I discovered that there were additional columns in the data set that provided text for the various encoded information.

3. There were many blank areas within the data set, and the data is mostly categorical in nature. I removed observations where there was no information on accident type or general causal factor

4. This data set was VERY messy. I definitely could have spent more time on actual analysis if I had a cleaner data set.


## Project Execution
This project required me to look at many different data sets from the NTSB, FAA, BTS, and NASA. I selected a dataset from the FAA's Accident/Incident Database (AIDS). This database is very thorough and spans many years (pre-1975, then each year through 2015). I began by looking at the data download site at http://av-info.faa.gov/dd_sublevel.asp?Folder=%5CAID.

The data is broken down by year groups, with one text file (tab delimited) per 5 year time period. The resulting data set is massive. I began by choosing the most recent year group (2015 - 2019) which includes incidents up to July 29th, 2017. Upon further analysis (~3 hours) of this file, I determined that many data fields I was interested in were missing, most likely due to recent changes in how the data is reported and stored in this database. I then looked back and a previous date range (2005-2009) and selected that file for further analysis. This file contains 15,782 rows and 180 columns.

I considered combining all data files into one DataFrame for analysis, but for the scope of this project, I chose to keep my analysis to a single file with incidents from 2005-2009. In the future, it would be interesting to conduct the same analysis on the combined data set to look at the trends of accidents and incidents over time.

In [ ]:
# Import sample data file from FAA Accident/Incident Database (AIDS):
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_rows', 500)
url = './data/A2005_09.txt'
faa_full = pd.read_table(url, sep='\t', parse_dates=True)

In [ ]:
faa_full.shape

# Data Familiarization and Munging Phase
### Getting Familiar With the Data
In this phase I examined basic information about the data set. After I pulled the data into my notebook, I found that I did not need most of the 180 columns. I imported the data dictionary from a text file into a spreadsheet (./data/Data Dictionary.xlsx) and looked at the columns to determine those I could drop for the main data set. I annotated the spreadsheet with my keep/drop decisions by coloring the features I wished to keep as green, and those to drop as red. I also categorized the features by pilot-related characteristics, aircraft-related characteristics, and added other notes. Finally, I added a column to the dictionary to sort it. This was necessary because I noticed that related features were not located next to each other in the downloaded data set. For example, the Primary Cause Factor text (c77) and Primary Cause Factor Code (c78) were stored as columns 146 and 122, respectively. Sorting the data dictionary allowed me to easily identify these related columns and extract only the features I wanted to analyze.

To facilitate the selection process I looked at value_counts() for each feature and compared the results to the Data Dictionary to get a feel for the data contained within and to further inform the features I would keep or drop. Based on sorting and analyzing value_counts, I realized that there were multiple columns that represented the same data. For example, several date and time fields were represented as both separate year, month, day, time columns and again as a combined column. Also, many of the columns are encoded (key is found in './data/AIDCODES.doc') with the same data recorded in plain text in another column. In these cases, I chose to keep the plain text and drop the encoded column.
### Munging
Once I completed my initial look at the data and the data dictionary, I went through the following steps to make it more manageable:
1. I dropped the columns I determined I would not need for this analysis.
2. I renamed the remaining columns to plain-language names
3. I dropped all rows that did not have any primary causal factors listed.
4. I dropped rows with no accident type listed.
5. After further analysis of the data, I decided to drop more columns. I originally had 42, and ultimately ended up with 15.

As some of the questions I wanted to examine would need some features while others would not, I kept the 'faa' dataset as whole as possible, and peeled off subsets of the data into feature sets as needed for my analysis.

### Final Data Features ~~(42)~~ (15)
| Column Name | Description |
| :------ | -------- | 
| ~~event_type~~ | ~~Type of Event~~ |
| date | Date the accident/incident happened |
| local_time | Local time of the accident/incident |
| num_engines | Number of engines on aircraft |
| ~~make~~ | ~~Make of the aircraft~~ |
| ~~model~~ | ~~Model of the aircraft~~ |
| ~~airframe_hours~~ | ~~Airframe hours of the aircraft~~ |
| ~~lat~~ | ~~Latitude coordinates of the accident/incident~~ |
| ~~long~~ | ~~Longitude coordinates of the accident/incident~~ |
| fatalities | Total Fatalities |
| injuries | All injuries including crew passengers and public |
| pilot_age | Age of the pilot in command |
| ~~pilot_total_hours_model~~ | ~~Total number of hours the pilot has flown in the specific make and model of the aircraft~~ | 
| ~~pilot_90_day_hours_model~~ | ~~Total number of hours in the last 90 days the pilot has flown this make and model~~ |
| ~~pilot_90_day_hours~~ | ~~Total number of hours in the last 90 days the pilot  has flown~~ |
| pilot_total_hours | Total number of hours the pilot has flown |
| flight_plan_desc | Flight plan description |
| pilot_cert | Certificate type text of the pilot in command |
| ~~pilot_qual~~ | ~~Qualification description of the pilot in command~~ |
| primary_cause | Primary cause factor text |
| ~~contributing_factor~~ | ~~Contributing factor text~~ |
| ~~persons_involved~~ | ~~Persons involved text~~ |
| ~~supporting_factor_technical~~ | ~~Supporting factor text - Technical~~ |
| ~~second_cause~~|~~Second cause factor text~~|
| ~~second_contributing~~ | ~~Second contributing factor text~~ |
| ~~second_persons~~ | ~~Second persons involved text~~ |
| ~~second_supporting_technical~~ | ~~Second supporting factor text - Technical~~ |
| accident_type | Type of accident or incident text |
| flight_phase | Phase of flight text |
| ~~damage~~ | ~~Damage text~~ |
| general_cause | General cause category text |
| primary_flying_conditions | Primary flying condition text |
| second_flying_conditions | Secondary flying condition text |
| ~~light_condition~~ | ~~Light condition text~~ |
| ~~wing_info~~ | ~~Wing information of the aircraft text~~ |
| ~~powered~~ | ~~Text for element C150 (Powered, Nonpowered, Optional)~~ |
| ~~engine_type~~ | ~~Type of engine~~ |
| ~~landing_gear~~ | ~~Type of landing gear~~ |
| ~~additional_cause~~ | ~~Additional cause factor text~~ |
| ~~2nd_additional_cause~~ | ~~2nd Additional cause factor text~~ |
| ~~supporting_factor_operational~~ | ~~Supporting factor Text - Operational~~ |
| ~~second_supporting_operational~~ | ~~Second supporting factor text - Operational~~|

In [ ]:
#Data Munging Phase

#Drop unwanted columns from dataset.

#I broke up the list of columns I wanted to drop as I determined them to reduce my chances of error.
droplist = ['c2','c3','c4','c5','c6','c7','c8','c11','c12','c13','c14','c15','c16','c17','c18','c19','c22','c25']
droplist = droplist + ['c26','c27','c30','c32','c33','c34','c35','c36','c37','c38','c39', 'c41','c43','c44','c45','c46','c47','c49']
droplist = droplist + ['c51','c52','c57','c58','c59','c61','c62','c63','c64','c65','c66','c67','c68','c69','c70','c71','c72','c73']
droplist = droplist + ['c74','c75','c78','c80','c82','c84','c86','c88','c90','c96','c98','c101','c102','c103','c104']
droplist = droplist + ['c106','c108','c110','c111','c112','c113','c114','c115','c117','c118','c119','c120','c121','c122','c123','c124']
droplist = droplist + ['c125','c126','c127','c128','c129','c131','c132','c133','c134','c135','c136','c137','c138','c139','c140']
droplist = droplist + ['c141','c143','c144','c145','c146','c147','c149','c152','c153','c154','c155','c157','c160','c162','c184','c192']
droplist = droplist + ['c203','c204','c205','c206','c207','c208','c210','c214','c229','c230','c233','c234','c240','c241','c242']
droplist = droplist + ['c243','c244','c790','end_of_record','c92','c94','c100']
faa_full = faa_full.drop(droplist, axis=1)

In [ ]:
faa_full.shape

In [ ]:
# I created plain-language column names that aligned with the data dictionary added them in the same order as the columns
columnlist=['event_type','date','local_time','num_engines','make','model','airframe_hours','lat','long']
columnlist = columnlist + ['fatalities','injuries','pilot_age','pilot_total_hours_model','pilot_90_day_hours_model']
columnlist = columnlist + ['pilot_90_day_hours','pilot_total_hours','flight_plan_desc','pilot_cert','pilot_qual']
columnlist = columnlist + ['primary_cause','contributing_factor','persons_involved','supporting_factor_technical']
columnlist = columnlist + ['second_cause','second_contributing','second_persons','second_supporting_technical']
columnlist = columnlist + ['accident_type','flight_phase','damage','general_cause','primary_flying_conditions']
columnlist = columnlist + ['second_flying_conditions','light_condition','wing_info','powered','engine_type']
columnlist = columnlist + ['landing_gear','additional_cause','2nd_additional_cause','supporting_factor_operational']
columnlist = columnlist + ['second_supporting_operational']
len(columnlist)

In [ ]:
faa_full.columns = columnlist # Renamed columns

In [ ]:
faa_full.head() # Sanity check on column labels

In [ ]:
# After further analysis, I returned to this portion of the notebook to drop more columns, as described above

droplist2 = ['event_type','make','model','airframe_hours','lat','long','contributing_factor','persons_involved']
droplist2 = droplist2 + ['supporting_factor_technical','second_cause','second_contributing','second_persons']
droplist2 = droplist2 + ['second_supporting_technical','damage','light_condition','wing_info','powered','engine_type']
droplist2 = droplist2 + ['landing_gear','additional_cause','2nd_additional_cause','supporting_factor_operational']
droplist2 = droplist2 + ['second_supporting_operational','pilot_total_hours_model','pilot_90_day_hours_model']
droplist2 = droplist2 + ['pilot_90_day_hours', 'pilot_qual']

In [ ]:
faa_full = faa_full.drop(droplist2, axis=1)

In [ ]:
faa_full.shape

In [ ]:
# When I started looking at the data, I realized there were many blanks, so I started by replacing all blank values

# In order to determine what exactly was in the blank values, I used:
# causes = pd.DataFrame(faa_full.general_cause.value_counts())
# causes.index
# Then I copied the number of spaces into the replace statement below:
# faa_full.replace('                    ', np.nan, inplace=True)

#Later, (after much pain) I found that I could strip all the whitespace across the DataFrame with a function:
def df_strip(df): 
  df = df.copy() 
  for c in df.columns:
    if df[c].dtype == np.object:
      df[c] = pd.core.strings.str_strip(df[c])
  return df

faa_full = df_strip(faa_full); # Calls above function to strip whitespace
# Then I could easily replace the blank cells with NaN, and drop any columns or rows easily.
faa_full.replace('', np.nan, inplace=True) # Replaces empty cells with NaN


In [ ]:
# Quickly drop all rows where there are no accident type or general cause listed
faa_full.dropna(subset=['general_cause','accident_type'], inplace=True)

In [ ]:
faa_full.shape

In [ ]:
faa_full.dtypes

In [ ]:
# replace "NaN" in object columns with "unknown" category
def df_unknown(df): 
  df = df.copy() 
  for c in df.columns:
    if df[c].dtype == np.object:
        df[c] = df[c].replace(np.nan, 'Unknown')
  return df

In [ ]:
faa_full.head()

In [ ]:
# Plot showing the top 10 types of accidents
top_accidents = faa_full.accident_type.value_counts().head(10).plot(kind='bar', figsize = (12,8))
top_accidents.set_xlabel('Accident Types', fontsize=16, labelpad=10)
top_accidents.set_ylabel('Number of Accidents', fontsize=16)
top_accidents.set_title('Top 10 Accidents by Type', fontsize=20, y=1.01)

In [ ]:
# Pull all accidents with fatalities
fatal_accidents = faa_full[faa_full['fatalities'] > 0].dropna()

In [ ]:
fatal_accidents.shape

In [ ]:
# Plot showing the top 10 fatal accidents, by # of occurrences
top_fatal = fatal_accidents.accident_type.value_counts().head(10).plot(kind='bar', figsize = (12,8))
top_fatal.set_xlabel('Accident Types', fontsize=16, labelpad=10)
top_fatal.set_ylabel('Number of Accidents', fontsize=16)
top_fatal.set_title('Top 10 Fatal Accidents by Type (Occurance)', fontsize=20, y=1.01)

In [ ]:
# Plot of top 10 fatal accident types, by total number of fatalities. Of note, The "Overshoot Landing" accident type
#     was only one event, which resulted in 49 fatalities.
fatal = fatal_accidents[['accident_type', 'fatalities']].groupby('accident_type').sum()\
                                                .sort_values('fatalities',ascending=False)\
                                                .head(10).plot(kind='bar', figsize=(12,8))
fatal.set_xlabel('Accident Types', fontsize=16, labelpad=10)
fatal.set_ylabel('Fatalities', fontsize=16)
fatal.set_title('Top 10 Fatal Accidents by Type (Fatalities)', fontsize=20, y=1.01)

In [ ]:
fatal_accidents['primary_cause'].value_counts().head(10)

In [ ]:
fatal_accidents['general_cause'].value_counts().head(10)

In [ ]:
# Pull all accidents with injuries
injury_accidents = faa_full[faa_full['injuries'] > 0].dropna()
injury_accidents.shape

In [ ]:
top_injury = injury_accidents[['accident_type','injuries']].groupby('accident_type').sum()\
                                              .sort_values('injuries', ascending=False)\
                                              .head(10).plot(kind='bar', figsize=(12,8))
top_injury.set_xlabel('Accident Types', fontsize=16, labelpad=10)
top_injury.set_ylabel('Injuries', fontsize=16)
top_injury.set_title('Top 10 Injurious Accidents by Type (Injuries)', fontsize=20, y=1.01)

In [ ]:
injury_accidents['primary_cause'].value_counts().head(10)

In [ ]:
injury_accidents['general_cause'].value_counts().head(10)

## Observations from EDA
Based on my inital examination of the data, I concluded that I needed to further simplify my analysis for this project. To that end, I decided to focus on those whether or not accidents were fatal or injurious, and try to relate that to pilot age, certification level, and total hours of flight time. (Because Pilot Induced accidents were the highest general cause for both fatal and injurious accidents)

In future study, I want to pay specific attention to Stalls and Spins, as those two departures from controlled flight can typically be avoided or recovered from with proper training and experience and when combined together are the most deadly types of accidents.

I created new columns (is_fatal) and (is_injurious) to represent whether or not an accident had fatalities or injuries.

In [ ]:
faa_full['is_fatal'] = faa_full['fatalities']>0
faa_full['is_injurious'] = faa_full['injuries']>0

In [ ]:
faa_full['is_fatal'].value_counts()

In [ ]:
faa_full['is_injurious'].value_counts()

In [ ]:
fatal_accidents.pilot_cert.value_counts()

In [ ]:
fatal_pilot_cert = fatal_accidents[['pilot_cert', 'fatalities']].groupby('pilot_cert').sum()\
                                                .sort_values('fatalities',ascending=False)\
                                                .head(10).plot(kind='bar', figsize=(12,8))
fatal_pilot_cert.set_xlabel('Pilot Certs', fontsize=16, labelpad=10)
fatal_pilot_cert.set_ylabel('Fatalities', fontsize=16)
fatal_pilot_cert.set_title('Top 10 Fatal Accidents by Pilot Certs', fontsize=20, y=1.01)

In [ ]:
fatal_accidents.plot(kind='scatter', x='pilot_total_hours', y='fatalities', ylim=(0,10), alpha=0.2)
#This suggests more fatalities where pilots have fewer total hours, but this could also be skewed
#  because there are fewer pilots and flights by pilots with a higher number of total hours.

In [ ]:
sns.swarmplot(x="fatalities", y="pilot_cert", data=fatal_accidents);

In [ ]:
sns.boxplot(x="fatalities", y="pilot_cert", data=fatal_accidents);

## Playing With Regressions
I attempted to model the fatal accidents data linearly, using the pilot's total hours and age as features, with the 
response variable being the number of fatalities.

I did the same with Logistic Regression and found that in both cases the model didn't work, as the number of fatalities depends on whether or not there are passengers on any given flight (among other factors). A better look at the data would be to consider whether or not fatalities or injuries occurred categorically, which I moved on to after experimenting with these regressions.

In [ ]:
from sklearn.linear_model import LinearRegression
LinReg = LinearRegression()
X = fatal_accidents[['pilot_total_hours', 'pilot_age']]
y = fatal_accidents.fatalities

X_train, X_test, y_train, y_test = train_test_split(X,y)


In [ ]:
LinReg.fit(X_train,y_train)
y_pred = LinReg.predict(X_test)
LinReg.score(X_test, y_test)

In [ ]:
# Logistic Regression was better than the terrible result above, but still bad
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
X = fatal_accidents[['pilot_total_hours', 'pilot_age']]
y = fatal_accidents.fatalities

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
LR.fit(X_train,y_train)
y_pred = LR.predict(X_test)
LR.score(X_test, y_test)

## Looking at Classification Trees
I decided I wanted to look at deciding how much different factors played in to whether or not an accident is fatal. To do this, I moved forward with the decision tree model after we learned about it in class.

I decided to look at the pilot's age, total number of flight hours, and level of certification to determine the response of whether or not the accident was fatal, and again to determine whether or not it was injurious.

To build the model I needed to build dummy variables for the pilot certification.

In [ ]:
#Build dataframe of dummy variables
pilot_cert_dummies = pd.get_dummies(faa_full.pilot_cert, prefix='Pilot_Cert')
pilot_cert_dummies.drop(pilot_cert_dummies.columns[0], axis=1, inplace=True)

In [ ]:
pilot_cert_dummies.columns.values

In [ ]:
#Concatenate the dummy variables on to full dataframe
faa_full = pd.concat([faa_full, pilot_cert_dummies], axis=1)

In [ ]:
#Get rid of NaN values in pilot hours and age by using median values
faa_full['pilot_total_hours'].fillna(faa_full['pilot_total_hours'].median(), inplace=True)
faa_full['pilot_age'].fillna(faa_full['pilot_age'].median(), inplace=True)


In [ ]:
# Begin building decision tree for fatal accidents
# Define X and y.

feature_cols = ['pilot_total_hours', 'pilot_age'] + list(pilot_cert_dummies.columns.values)

X = faa_full[feature_cols]
y = faa_full.is_fatal

In [ ]:
# Fit a classification tree with max_depth=3 on all data.
from sklearn.tree import DecisionTreeClassifier

treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf.fit(X, y)

In [ ]:
from sklearn.tree import export_graphviz
export_graphviz(treeclf, out_file='./data/tree_fatalities.dot', feature_names=feature_cols);

![title](./data/tree_fatalities.png)

In [ ]:
# Looking at the feature importance
pd.DataFrame({'feature':feature_cols, 'importance':treeclf.feature_importances_})

In [ ]:
# Now I'll try a Decision Tree without the pilot certification level.
feature_cols = ['pilot_total_hours', 'pilot_age']
X = faa_full[feature_cols]
y = faa_full.is_fatal

treeclf2 = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf2.fit(X, y)


In [ ]:
export_graphviz(treeclf2, out_file='./data/tree_fatalities2.dot', feature_names=feature_cols);

![title](./data/tree_fatalities2.png)

In [ ]:
# Another look at feature importance
pd.DataFrame({'feature':feature_cols, 'importance':treeclf2.feature_importances_})

In [ ]:
# Begin building decision tree for injurious accidents
# Define X and y.

feature_cols = ['pilot_total_hours', 'pilot_age'] + list(pilot_cert_dummies.columns.values)

X = faa_full[feature_cols]
y = faa_full.is_injurious

treeclf3 = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf3.fit(X, y)

export_graphviz(treeclf3, out_file='./data/tree_injuries.dot', feature_names=feature_cols);

![title](./data/tree_injuries.png)

In [ ]:
# The data suggests that pilot age is significant in determining likelihood of an accident resulting in injury
pd.DataFrame({'feature':feature_cols, 'importance':treeclf3.feature_importances_})

In [ ]:
# Another look, removing the certification level
feature_cols = ['pilot_total_hours', 'pilot_age']

X = faa_full[feature_cols]
y = faa_full.is_injurious

treeclf4 = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf4.fit(X, y)

export_graphviz(treeclf4, out_file='./data/tree_injuries2.dot', feature_names=feature_cols);

![title](./data/tree_injuries2.png)

In [ ]:
pd.DataFrame({'feature':feature_cols, 'importance':treeclf4.feature_importances_})